In [0]:
%pip install mlflow==2.10.1 langchain databricks-vectorsearch==0.22 databricks-sdk==0.18.0 mlflow[databricks]
dbutils.library.restartPython()

In [0]:
%run ./00-init $reset_all_data=false

In [0]:
index_name=f"{catalog}.{db}.vector_search_index"
host = "https://" + spark.conf.get("spark.databricks.workspaceUrl")

In [0]:
# url used to send the request to your model from the serverless endpoint
import os
host = "https://" + spark.conf.get("spark.databricks.workspaceUrl")
os.environ['DATABRICKS_TOKEN'] = dbutils.secrets.get('databricksscope', 'databricks-token') 
#os.environ['DATABRICKS_TOKEN'] = ""

In [0]:
from databricks.vector_search.client import VectorSearchClient
from langchain_community.vectorstores import DatabricksVectorSearch
from langchain_community.embeddings import DatabricksEmbeddings

# Test embedding Langchain model
#NOTE: your question embedding model must match the one used in the chunk in the previous model 
embedding_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")
print(f"Test embeddings: {embedding_model.embed_query('What is Apache Spark?')[:20]}...")

def get_retriever(persist_dir: str = None):
    os.environ["DATABRICKS_HOST"] = host
    #Get the vector search index
    vsc = VectorSearchClient(workspace_url=host, personal_access_token=os.environ["DATABRICKS_TOKEN"])
    vs_index = vsc.get_index(
        endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
        index_name=index_name
    )

    # Create the retriever
    vectorstore = DatabricksVectorSearch(
        vs_index, text_column="content", embedding=embedding_model
    )
    return vectorstore.as_retriever()

# test our retriever
vectorstore = get_retriever()
similar_documents = vectorstore.get_relevant_documents("What is the current Market Sentiment for Contoso following its acquisition of Litware and explain the reasons for it?")
print(f"Relevant documents: {similar_documents[0]}")

Test embeddings: [0.0185699462890625, -0.01403045654296875, -0.057647705078125, 0.003448486328125, 0.008575439453125, -0.0216827392578125, -0.0247344970703125, -0.0047149658203125, 0.0136260986328125, 0.050323486328125, -0.027496337890625, -0.0147247314453125, 0.05474853515625, -0.053802490234375, -0.01025390625, -0.0161895751953125, -0.018768310546875, -0.017181396484375, -0.051177978515625, 0.0178680419921875]...
[NOTICE] Using a Personal Authentication Token (PAT). Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


/databricks/python/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Relevant documents: page_content="Quarterly Market Sentiment Report March 2024\nSummary:\nContoso, a retail company headquartered in the US, took a strategic step to improve its market sentiment and customer satisfaction by acquiring LitWare, a European retail company based in the UK. This acquisition has significantly enhanced Contoso's market sentiment score, which has now risen to an impressive 70%. Moreover, the acquisition has resulted in high customer satisfaction and positive reviews, marking a notable turnaround in Contoso's performance.\n© 2023 Contoso Confidential\nPrior to the acquisition, Contoso faced challenges with a market sentiment that was below average compared to industry trends. The lack of focus on Environmental, Social, and Governance (ESG) initiatives led to poor customer satisfaction and lower market sentiment ratings, especially among Millennials, a key market segment for Contoso.\nHowever, with the integration of LitWare, Contoso has successfully addressed th

In [0]:
# Test Databricks Foundation LLM model
from langchain_community.chat_models import ChatDatabricks
chat_model = ChatDatabricks(endpoint="databricks-dbrx-instruct", max_tokens = 2000)
print(f"Test chat model: {chat_model.predict('What is the current Market Sentiment for Contoso following its acquisition of Litware and explain the reasons for it?')}")

Test chat model: Based on the information available up to December 2023, the market sentiment for Contoso following its acquisition of Litware is positive. This is due to several reasons:

1. Increased Market Share: The acquisition of Litware has allowed Contoso to expand its market share in the technology sector, making it a more dominant player in the industry.

2. Diversification of Products and Services: Litware's product and service offerings complement those of Contoso, allowing the combined entity to offer a more diverse range of solutions to its customers.

3. Economies of Scale: The merger of the two companies is expected to result in significant cost savings due to economies of scale, which is viewed positively by the market.

4. Enhanced Competitive Position: The acquisition has strengthened Contoso's competitive position, enabling it to better compete with other major players in the technology sector.

5. Improved Financial Performance: The market expects the acquisition to

In [0]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatDatabricks

TEMPLATE = """You are an assistant for Databricks users. You are answering python, coding, SQL, data engineering, spark, data science, DW and platform, API or infrastructure administration question related to Databricks. If the question is not related to one of these topics, kindly decline to answer. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.
Use the following pieces of context to answer the question at the end:
{context}
Question: {question}
Answer:
"""
prompt = PromptTemplate(template=TEMPLATE, input_variables=["context", "question"])

chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=get_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

[NOTICE] Using a Personal Authentication Token (PAT). Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


In [0]:
# langchain.debug = True #uncomment to see the chain details and the full prompt being sent
question = {"query": "What is the current Market Sentiment for Contoso following its acquisition of Litware and explain the reasons for it?"}
answer = chain.run(question)
print(answer)


/databricks/python/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The current Market Sentiment for Contoso is at an impressive 70%, which is a significant improvement from the previous score of 40%. This improvement can be attributed to Contoso's strategic acquisition of LitWare, a European retail company based in the UK. The acquisition has allowed Contoso to enhance its market sentiment by addressing concerns related to Environmental, Social, and Governance (ESG) initiatives, which were previously lacking and contributing to poor customer satisfaction.

The integration of LitWare has led to a remarkable improvement in customer satisfaction, with customers expressing extreme happiness and satisfaction with their purchases. The product quality, performance, and customer service provided by Contoso post-acquisition have exceeded expectations, leading to highly positive feedback from consumers. Customers highlight the exceptional quality of products, prompt and helpful customer service, and overall fantastic experience with Contoso's offerings. These p

In [0]:
from mlflow.models import infer_signature
import mlflow
import langchain

mlflow.set_registry_uri("databricks-uc")
model_name = f"{catalog}.{db}.rag_chatbot_model"

with mlflow.start_run(run_name="chatbot_rag") as run:
    signature = infer_signature(question, answer)
    model_info = mlflow.langchain.log_model(
        chain,
        loader_fn=get_retriever,  # Load the retriever with DATABRICKS_TOKEN env as secret (for authentication).
        artifact_path="chain",
        registered_model_name=model_name,
        pip_requirements=[
            "mlflow==" + mlflow.__version__,
            "langchain==" + langchain.__version__,
            "databricks-vectorsearch",
        ],
        input_example=question,
        signature=signature
    )

2024/06/06 13:10:24 INFO mlflow.models.utils: We convert input dictionaries to pandas DataFrames such that each key represents a column, collectively constituting a single row of data. If you would like to save data as multiple rows, please convert your data to a pandas DataFrame before passing to input_example.
/databricks/python/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Uploading artifacts:   0%|          | 0/7 [00:00<?, ?it/s]

Successfully registered model 'litware_unity_catalog.rag.rag_chatbot_model'.


Uploading artifacts:   0%|          | 0/7 [00:00<?, ?it/s]

Created version '1' of model 'litware_unity_catalog.rag.rag_chatbot_model'.


In [0]:
model = mlflow.langchain.load_model(model_info.model_uri)
model.invoke(question)

[NOTICE] Using a Personal Authentication Token (PAT). Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


/databricks/python/lib/python3.11/site-packages/langchain_core/utils/loading.py:22: UserWarning: Loading from the deprecated github-based Hub is no longer supported. Please use the new LangChain Hub at https://smith.langchain.com/hub instead.
  warnings.warn(
/databricks/python/lib/python3.11/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(


{'query': 'What is the current Market Sentiment for Contoso following its acquisition of Litware and explain the reasons for it?',
 'result': "The current Market Sentiment for Contoso is at an impressive 70%, which is a significant improvement from the previous score of 40%. This improvement can be attributed to Contoso's strategic acquisition of LitWare, a European retail company based in the UK. The acquisition has allowed Contoso to enhance its market sentiment by addressing concerns related to Environmental, Social, and Governance (ESG) initiatives, which were previously lacking and contributing to poor customer satisfaction.\n\nThe integration of LitWare has led to a remarkable improvement in customer satisfaction, with customers expressing extreme happiness and satisfaction with their purchases. The product quality, performance, and customer service provided by Contoso post-acquisition have exceeded expectations, leading to highly positive feedback from consumers. Customers highl

### Deploying our Chat Model as a Serverless Model Endpoint


In [0]:
import mlflow
from mlflow.tracking import MlflowClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedModelInput, ServedModelInputWorkloadSize

In [0]:
mlflow.set_registry_uri('databricks-uc')
model_name = f"{catalog}.{db}.rag_chatbot_model"
serving_endpoint_name = "rag-chatbot-model-endpoint"
latest_model_version = 1

In [0]:
# Create or update serving endpoint
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedModelInput, ServedModelInputWorkloadSize

endpoint_config = EndpointCoreConfigInput(
    name=serving_endpoint_name,
    served_models=[
        ServedModelInput(
            model_name=model_name,
            model_version=latest_model_version,
            workload_size=ServedModelInputWorkloadSize.LARGE,
            scale_to_zero_enabled=True,
            environment_vars={
                "DATABRICKS_TOKEN": "{{secrets/databricksscope/databricks-token}}" ,
            }
        )
    ]
)

In [0]:
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()

existing_endpoint = next(
    (e for e in w.serving_endpoints.list() if e.name == serving_endpoint_name), None
)
serving_endpoint_url = f"{host}/ml/endpoints/{serving_endpoint_name}"
if existing_endpoint == None:
    print(f"Creating the endpoint {serving_endpoint_url}, this will take a few minutes to package and deploy the endpoint...")
    w.serving_endpoints.create_and_wait(name=serving_endpoint_name, config=endpoint_config)
else:
    print(f"Updating the endpoint {serving_endpoint_url} to version {latest_model_version}, this will take a few minutes to package and deploy the endpoint...")
    w.serving_endpoints.update_config_and_wait(served_models=endpoint_config.served_models, name=serving_endpoint_name)

Creating the endpoint https://adb-3377374060933843.3.azuredatabricks.net/ml/endpoints/rag-chatbot-model-endpoint, this will take a few minutes to package and deploy the endpoint...
